In [151]:
import pandas as pd
import requests
from matplotlib import pyplot as plt
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA

In [152]:
!pip install fbprophet

  Using cached https://files.pythonhosted.org/packages/f7/86/4509e952f9724f084625e93e0bf8d8519b25c79029a0a916b0f996644c75/fbprophet-0.6.tar.gz
  ERROR: Complete output from command /snap/jupyter/6/bin/python -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-3g07xms2/fbprophet/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /tmp/pip-wheel-leqf9d_r --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/fbprophet
  creating build/lib/fbprophet/stan_model
  Importing plotly failed. Interactive plots will not work.
  INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_888a84912910fa0a45b9d614b75bb8a8 NOW.
  error: command 'gcc' failed with exit status 1
  ----------------------------------------
  ERROR: Failed building wheel for fbprophet
  Runn

In [153]:
url = 'https://min-api.cryptocompare.com/data/v2/histoday?fsym=BTC&tsym=USD&limit=1000'

In [154]:
def call(url):
    ''' 
    funcion que llama a la api de cryptocompare

    '''
    response = requests.get(url)
    read = response.json()
    return read

In [155]:
def crypto_definer(url):
    ''' Extrae los valores de la crypto y los pega en un dataframe '''
    moneda = call(url)
    high = []
    low = []
    Open=[]
    close = []
    unx_time= []
    for i in moneda['Data']['Data']:
        unx_time.append(i['time'])
        high.append(i['high'])
        low.append(i['low'])
        Open.append(i['open'])
        close.append(i['close'])
    df = pd.DataFrame()
    df['UNIX_TIME'] = unx_time
    df['Open'] = Open
    df['High'] = high
    df['Low'] = low
    df['Close'] = close
    return df

In [156]:
crypto_definer(url)[500:]

,UNIX_TIME,Open,High,Low,Close
500,1539302400,6209.47,6296.56,6173.86,6250.85
501,1539388800,6252.10,6280.73,6238.92,6267.27
502,1539475200,6267.27,6346.56,6247.17,6277.73
503,1539561600,6277.73,7234.83,6244.77,6612.36
504,1539648000,6612.69,6724.91,6545.54,6583.05
...,...,...,...,...,...
996,1582156800,9602.43,9698.59,9402.81,9608.80
997,1582243200,9608.80,9766.16,9582.07,9697.86
998,1582329600,9697.86,9720.21,9577.84,9671.46
999,1582416000,9671.46,10011.40,9669.88,9977.39


In [157]:
def convert(url):
    df = crypto_definer(url)
    df['UNIX_TIME'] = df['UNIX_TIME'].astype(int)
    df['UNIX_TIME'] = pd.to_datetime(df['UNIX_TIME'], unit='s')

    return df

In [158]:
def df_y(url, columna='High'):
    """ Convierte las columnas de time y precio X a un formato predeterminado
     que pide facebook prophet, la fecha como DS y la columna a predecir como y """
    df = convert(url)
    new_df = df.rename(columns = {'UNIX_TIME': 'ds', columna : 'y'})
    new_df = new_df[['ds', 'y']]
    return new_df

In [159]:
df_high = df_y(url)
df_open = df_y(url, 'Open')
df_low = df_y(url, 'Low')
df_close = df_y(url, 'Close')

In [160]:
df_high[500:]

,ds,y
500,2018-10-12,6296.56
501,2018-10-13,6280.73
502,2018-10-14,6346.56
503,2018-10-15,7234.83
504,2018-10-16,6724.91
...,...,...
996,2020-02-20,9698.59
997,2020-02-21,9766.16
998,2020-02-22,9720.21
999,2020-02-23,10011.40


In [161]:
df_open[500:]

,ds,y
500,2018-10-12,6209.47
501,2018-10-13,6252.10
502,2018-10-14,6267.27
503,2018-10-15,6277.73
504,2018-10-16,6612.69
...,...,...
996,2020-02-20,9602.43
997,2020-02-21,9608.80
998,2020-02-22,9697.86
999,2020-02-23,9671.46


In [162]:
df_low[500:]

,ds,y
500,2018-10-12,6173.86
501,2018-10-13,6238.92
502,2018-10-14,6247.17
503,2018-10-15,6244.77
504,2018-10-16,6545.54
...,...,...
996,2020-02-20,9402.81
997,2020-02-21,9582.07
998,2020-02-22,9577.84
999,2020-02-23,9669.88


In [163]:
df_close[500:]

,ds,y
500,2018-10-12,6250.85
501,2018-10-13,6267.27
502,2018-10-14,6277.73
503,2018-10-15,6612.36
504,2018-10-16,6583.05
...,...,...
996,2020-02-20,9608.80
997,2020-02-21,9697.86
998,2020-02-22,9671.46
999,2020-02-23,9977.39


In [164]:
train_high, test_high=df_high['y'][:500], df_high['y'][500:]
train_open, test_open=df_high['y'][:500], df_high['y'][500:]
train_low, test_low=df_high['y'][:500], df_high['y'][500:]
train_close, test_low=df_high['y'][:500], df_high['y'][500:]

In [165]:
modelo_high=ARIMA(train_high, order=(20,2,5)).fit(method='mle')
modelo_open=ARIMA(train_open,order=(20,2,5)).fit(method='mle')
modelo_low=ARIMA(train_low,order=(20,2,5)).fit(method='mle')
modelo_close=ARIMA(train_close,order=(20,2,5)).fit(method='mle')

ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.

In [ ]:
pred_high = modelo.predict(len(train_high), len(df_hight['y']))
pred_open = modelo.predict(len(train_open), len(df_open['y']))
pred_low = modelo.predict(len(train_low), len(df_low['y']))
pred_close = modelo.predict(len(train_close), len(df_close['y']))

In [ ]:
pred_high

In [ ]:
pred_open

In [ ]:
pred_low

In [ ]:
pred_close

In [ ]:
error=(pred-test).abs().sum()/len(pred)
error

In [ ]:
res=pd.DataFrame({'real':test, 'prediccion':pred, 'diff':pred-test})
res[:-1]

In [ ]:
new_df = df[500:].join(res[:-1])
new_df[148:].head(10)

In [ ]:
f

In [ ]:
for i in res['prediccion'][:-1]:
    print(i)

In [ ]:
res.prediccion.index

In [ ]:
plt.plot(res.prediccion,res.index)